# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
import requests
import pandas as pd
import os

In [3]:
# Using Foursquare API credentials
FOURSQUARE_API_KEY = os.environ['foursquare_api']  # using Foursquare API key
VERSION = "20230824"  # Use the current date for the version (e.g., 20230822)

In [4]:
dfCB=pd.read_csv("../data/city_bikes.csv")
dfCB

,latitude,longitude,num_bikes
0,40.795000,-73.964500,26
1,40.766741,-73.979069,24
2,40.676368,-73.952918,2
3,40.751047,-73.937970,13
4,40.801694,-73.957145,24
...,...,...,...
1974,40.752430,-73.867170,2
1975,40.766460,-73.886740,14
1976,40.751380,-73.862420,1
1977,40.757550,-73.864540,3


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [6]:
#----foursquare
for idx in dfCB.index:
    latitude= dfCB.loc[idx,"latitude"]
    longitude= dfCB.loc[idx,"longitude"]
    #custom_function(latitude,longitude)
    url = "https://api.foursquare.com/v3/places/search"
    # Define the query parameters and pass it in
    headers = {"accept": "application/json",
            "authorization":FOURSQUARE_API_KEY}
    params = {
    "v": VERSION,
    "ll": f"{city_latitude},{city_longitude}",  # City's latitude and longitude
    "radius": 1000,  # Radius in meters (1000m = 1km)
    "categoryId": "13065",  # Foursquare category ID for bike rental / bike share
    "limit": 50
    }
    response = requests.get(url, headers=headers, params=params)
    venue_data = []  # List to store venue details
   # if response.status_code == 200:
    data = response.json()
        
   # else:
    #    print("Request failed with status code:", response.status_code)
    venues = data['results']  # Extract the list of venues
    for venue in venues:
        venue_name = venue['name']
        category = venue['categories'][0]['name'] if 'categories' in venue and len(venue['categories']) > 0 else "Unknown"
        rating = venue['rating'] if 'rating' in venue else "N/A"
        latitude = venue['geocodes']['main']['latitude']
        longitude = venue['geocodes']['main']['longitude']
        venue_details = {
            "Venue": venue_name,
            "Category": category,
            "Rating": rating,
            "Latitude": latitude,
            "Longitude": longitude
        }
        venue_data.append(venue_details)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

Put your parsed results into a DataFrame

In [ ]:
venue_data

In [ ]:
# Create a DataFrame from the venue_data list
dffoursquare = pd.DataFrame(venue_data)
dffoursquare

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests
import pandas as pd
import os

# Using Foursquare API credentials
YELP_API_KEY = os.environ['API_YELP']  # using Foursquare API key
URL = "https://api.yelp.com/v3/businesses/search"

# City coordinates (latitude and longitude)
city_latitude = 55.7449006
city_longitude = 37.6020183  

# Set the radius (1000 meters)
radius = 1000

# Define headers with the API key
headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}

In [ ]:
# Parameters for the API request
params = {
    "latitude": city_latitude,
    "longitude": city_longitude,
    "radius": radius,
    "categories": "bikes",
    "limit": 50  # Number of results to retrieve (adjust as needed)
}

# Send GET request to Yelp API
response = requests.get(URL, headers=headers, params=params)
response

# Process the response
if response.status_code == 200:
    data = response.json()
    bike_stations = data['businesses']
    for station in bike_stations:
        station_name = station['name']
        location = station['coordinates']  # The 'coordinates' field contains latitude and longitude
        latitude = location['latitude']
        longitude = location['longitude']
        
        print("Station:", station_name)
        print("Latitude:", latitude)
        print("Longitude:", longitude)
        print("---")
else:
    print("Request failed with status code:", response.status_code)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)
# Parameters for the API request
params = {
    "latitude": city_latitude,
    "longitude": city_longitude,
    "radius": radius,
    "categories": "restaurants,bars",  # Categories you're interested in (add more if needed)
    "limit": 50  # Number of results to retrieve (adjust as needed)
}

In [ ]:
# Process the response
if response.status_code == 200:
    data = response.json()
    pois = data['businesses']
    
    for poi in pois:
        name = poi['name']
        rating = poi.get('rating', "N/A")
        location = poi.get('location', {})
        address = ', '.join(location.get('display_address', []))
        
        print("Name:", name)
        print("Rating:", rating)
        print("Address:", address)
        print("---")
else:
    print("Request failed with status code:", response.status_code)

Put your parsed results into a DataFrame

In [ ]:
# parse results into a DataFrame
# Process the response
if response.status_code == 200:
    data = response.json()
    pois = data['businesses']
    
    # Create a list to store parsed POI data
    parsed_pois = []
    
    for poi in pois:
        name = poi['name']
        rating = poi.get('rating', "N/A")
        location = poi.get('location', {})
        address = ', '.join(location.get('display_address', []))
        
        parsed_pois.append({
            'Name': name,
            'Rating': rating,
            'Address': address
        })

    # Create a DataFrame from the parsed POI data
    df = pd.DataFrame(parsed_pois)
    print(df)
    
else:
    print("Request failed with status code:", response.status_code)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

When it comes to broad range of places, venues, and points of interest, Foursquare API is a good option. It covers various categories, making it suitable for finding popular attractions, restaurants, shops, and more. However, If the focus is on food, dining, and local businesses, the Yelp Fusion API is well-suited for finding restaurants, cafes, bars, and related services. It provides comprehensive information about businesses, reviews, and ratings in the hospitality sector.
In my own opinion the Yelp Fusion API provided more detail on ratings when compared to the Foursquare API.

Get the top 10 restaurants according to their rating

In [ ]:
# Process the response
if response.status_code == 200:
    data = response.json()
    top_restaurants = data['businesses'][:10]  # Get the top 10 restaurants
    
    # Create a list to store parsed restaurant data
    parsed_restaurants = []
    
    for restaurant in top_restaurants:
        name = restaurant['name']
        rating = restaurant.get('rating', "N/A")
        location = restaurant.get('location', {})
        address = ', '.join(location.get('display_address', []))
        
        parsed_restaurants.append({
            'Name': name,
            'Rating': rating,
            'Address': address
        })

    # Create a DataFrame from the parsed restaurant data
    df = pd.DataFrame(parsed_restaurants)
    print(df)
    
else:
    print("Request failed with status code:", response.status_code)